## Instructor and responsemodel

In [15]:
# built-in libraries
from typing import TypeVar, Literal, Any

# litellm libraries
import litellm
from litellm import completion
from instructor import Mode, from_litellm

# misc libraries
from decouple import config
from pydantic import BaseModel, Field, create_model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

In [6]:
# Load data
df = pd.read_csv("data/oxford_ekman_emotions.csv")

# Display a sample
df.sample(5)

,emotion,definition
0,Anger,"A strong feeling of annoyance, displeasure, or..."
4,Sadness,The condition or quality of being sad; sorrow;...
2,Fear,An unpleasant emotion caused by the threat of ...
1,Disgust,A strong feeling of dislike or disapproval for...
3,Joy,A feeling of great pleasure and happiness.


setting up the watsonx credentials


In [7]:
# Create emotion definitions
emotion_definitions = dict(zip(df['emotion'], df['definition']))

for emotion, definition in emotion_definitions.items():
    print(f"{emotion.upper()}: {definition}\n")


ANGER: A strong feeling of annoyance, displeasure, or hostility.

DISGUST: A strong feeling of dislike or disapproval for something unpleasant or offensive.

FEAR: An unpleasant emotion caused by the threat of danger, pain, or harm.

JOY: A feeling of great pleasure and happiness.

SADNESS: The condition or quality of being sad; sorrow; a feeling of unhappiness or grief.

SURPRISE: A feeling of mild astonishment or shock caused by something unexpected.

NEUTRAL: Not displaying any strong emotion or feeling; a lack of emotional expression.



In [26]:
from litellm import completion
from dotenv import load_dotenv
import os

load_dotenv()

response = completion(
    model="watsonx/mistralai/mistral-large",
    messages=[{"role": "user", "content": "what is your favorite colour?"}],
    api_key=os.getenv("WX_API_KEY"),
    api_base=os.getenv("WX_URL"),
    project_id=os.getenv("WX_PROJECT_ID_RAG"),
    stream=True
)

# Only print the generated content, not the full object
for chunk in response:
    content = chunk.choices[0].delta.content
    if content:
        print(content, end="")



 I don't have personal preferences or favorites, as I'm a text-based AI and don't have feelings or consciousness. However, I can tell you that according to a survey by YouGov in 2019, the most popular color among Americans was blue. If you'd like to share your favorite color, I'd be happy to know!

In [27]:
class EmotionInstructor:
    def __init__(self, llm, definitions=None):
        self.llm = llm
        self.definitions = definitions or {}
        self.system_prompt = self._build_system_prompt()

    def _build_system_prompt(self):
        intro = "You are an expert in detecting human emotions from customer reviews using Ekman's 7 emotions.\n"
        instructions = "Use ONLY the following definitions:\n\n"
        for emotion, definition in self.definitions.items():
            instructions += f"{emotion.upper()}: {definition}\n"
        return intro + instructions

    def predict_emotion(self, review_text):
        prompt = (
            f"Review: \"{review_text}\"\n\n"
            "Identify all applicable emotions expressed in the review, based on Ekman's 7 emotions. "
            "Return only the emotion names as a comma-separated list. Return 'neutral' if no emotion applies."
        )

        response = self.llm.chat(
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=50
        )

        # Clean up response and return a list of emotions
        content = response["choices"][0]["message"]["content"]
        return [e.strip().lower() for e in content.split(",") if e.strip()]




In [28]:
# Assuming you have this already:
# emotion_definitions = dict(zip(df['emotion'], df['definition']))

instructor = EmotionInstructor(llm=llm, definitions=emotion_definitions)

review = "I am scared and deeply upset by how I was treated."
print(instructor.predict_emotion(review))



NameError: name 'llm' is not defined

In [46]:
import pandas as pd

# Load the test set and take a random sample of 500
test_df = pd.read_csv("data/ekman_test.csv").sample(n=500, random_state=42)

# Predict emotions
test_df["predicted_emotion"] = test_df["text"].apply(instructor.predict_emotion)

# Save the results
test_df.to_csv("ekman_test_with_predictions_sampled.csv", index=False)


18:27:30 - LiteLLM:WARNING: utils.py:511 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0.3, 'max_tokens': 50, 'project_id': '5b56ca80-258f-4dca-b3b9-413dd0495235'}


18:27:33 - LiteLLM:WARNING: utils.py:511 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-6da81cad48ca4d0810a4045b5eeb1b36", "object": "chat.completion", "model_id": "mistralai/mistral-large", "model": "mistralai/mistral-large", "choices": [{"index": 0, "message": {"role": "assistant", "content": " SURPRISE"}, "finish_reason": "stop"}], "created": 1746548853, "model_version": "2.0.0", "created_at": "2025-05-06T16:27:33.350Z", "usage": {"completion_tokens": 5, "prompt_tokens": 240, "total_tokens": 245}, "system": {"warnings": [{"message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.", "id": "disclaimer_warning", "more_info": "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx"}]}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0.3, 'max_tokens': 50, 'project_id

18:27:33 - LiteLLM:WARNING: utils.py:511 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

RAW RESPONSE:
{"id": "chatcmpl-7d0d981da1401676c47d540ab6c26222", "object": "chat.completion", "model_id": "mistralai/mistral-large", "model": "mistralai/mistral-large", "choices": [{"index": 0, "message": {"role": "assistant", "content": " SURPRISE,JOY"}, "finish_reason": "stop"}], "created": 1746549153, "model_version": "2.0.0", "created_at": "2025-05-06T16:32:34.177Z", "usage": {"completion_tokens": 8, "prompt_tokens": 243, "total_tokens": 251}, "system": {"warnings": [{"message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.", "id": "disclaimer_warning", "more_info": "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx"}]}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0.3, 'max_tokens': 50, 'projec

18:32:34 - LiteLLM:WARNING: utils.py:511 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-0ee0eb88ab465517d894c836817606e3", "object": "chat.completion", "model_id": "mistralai/mistral-large", "model": "mistralai/mistral-large", "choices": [{"index": 0, "message": {"role": "assistant", "content": " JOY, NEUTRAL"}, "finish_reason": "stop"}], "created": 1746549154, "model_version": "2.0.0", "created_at": "2025-05-06T16:32:34.766Z", "usage": {"completion_tokens": 8, "prompt_tokens": 257, "total_tokens": 265}, "system": {"warnings": [{"message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.", "id": "disclaimer_warning", "more_info": "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx"}]}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0.3, 'max_tokens': 50, 'projec

18:32:35 - LiteLLM:WARNING: utils.py:511 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-164f8d598f2789fceec599a6da0133ac", "object": "chat.completion", "model_id": "mistralai/mistral-large", "model": "mistralai/mistral-large", "choices": [{"index": 0, "message": {"role": "assistant", "content": " ANGER, JOY"}, "finish_reason": "stop"}], "created": 1746549155, "model_version": "2.0.0", "created_at": "2025-05-06T16:32:35.253Z", "usage": {"completion_tokens": 6, "prompt_tokens": 254, "total_tokens": 260}, "system": {"warnings": [{"message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.", "id": "disclaimer_warning", "more_info": "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx"}]}}


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0.3, 'max_tokens': 50, 'project_